In [1]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
train_dir = '/Users/suparnac/dev_envs/CV_Projects/Amarican_Sign_Language/dataset_ASL/asl_alphabet_train/asl_alphabet_train'
test_dir = '/Users/suparnac/dev_envs/CV_Projects/Amarican_Sign_Language/dataset_ASL/asl_alphabet_test/asl_alphabet_test'

In [3]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # For RGB images
])


In [4]:
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

print(f'Number of training samples: {len(train_dataset)}')
print(f'Number of test samples: {len(test_dataset)}')
print(f'Class to index mapping: {train_dataset.class_to_idx}')


Number of training samples: 87001
Number of test samples: 1
Class to index mapping: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, 'del': 26, 'nothing': 27, 'space': 28}


In [5]:
import torch
from torchvision import models
import torch.nn as nn

In [ ]:
# Set device (to MPS)
device = torch.device("mps")
print(f"Using device: {device}")


In [7]:
# Load pretrained ResNet18 and modify final layer for 29 classes
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 29)

# Move model to the device
model = model.to(device)

print(model)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

/Users/suparnac/dev_envs/cv_env/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/suparnac/dev_envs/cv_env/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
import torch.optim as optim

In [9]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [10]:
# Training loop for one epoch
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc


In [11]:
# Quick check with 1 epoch
train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}')


Train Loss: 0.1027, Train Accuracy: 0.9693


In [12]:
from torch.utils.data import random_split

In [13]:
def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc


In [14]:
def train_model_early_stopping(model, train_loader, val_loader, criterion, optimizer, device, epochs=10, patience=3):
    best_val_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(epochs):
        train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        
        print(f'Epoch {epoch+1}/{epochs} - '
              f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} || '
              f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
        
        # Early stopping check
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'best_model_asl.pth') 
        else:
            patience_counter += 1
            
        if patience_counter >= patience:
            print(f"Early stopping triggered after {patience} epochs with no improvement.")
            break


In [15]:
val_size = int(0.1 * len(train_dataset)) #10% validation
train_size = len(train_dataset) - val_size
train_subset, val_subset = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)


In [16]:
# training the model
train_model_early_stopping(model, train_loader, val_loader, criterion, optimizer, device, epochs=10, patience=3)

Epoch 1/10 - Train Loss: 0.0377, Train Acc: 0.9890 || Val Loss: 0.0094, Val Acc: 0.9969
Epoch 2/10 - Train Loss: 0.0207, Train Acc: 0.9941 || Val Loss: 0.0053, Val Acc: 0.9993
Epoch 3/10 - Train Loss: 0.0208, Train Acc: 0.9946 || Val Loss: 0.0031, Val Acc: 0.9992
Epoch 4/10 - Train Loss: 0.0160, Train Acc: 0.9956 || Val Loss: 0.0218, Val Acc: 0.9936
Epoch 5/10 - Train Loss: 0.0130, Train Acc: 0.9966 || Val Loss: 0.0006, Val Acc: 0.9998
Epoch 6/10 - Train Loss: 0.0118, Train Acc: 0.9969 || Val Loss: 0.0028, Val Acc: 0.9991
Epoch 7/10 - Train Loss: 0.0101, Train Acc: 0.9974 || Val Loss: 0.0129, Val Acc: 0.9952
Epoch 8/10 - Train Loss: 0.0070, Train Acc: 0.9982 || Val Loss: 0.0014, Val Acc: 0.9994
Early stopping triggered after 3 epochs with no improvement.
